# Sistema de Triagem e Classificação de Currículos

## 1. Visão Geral
Este sistema implementa uma solução automatizada para triagem e classificação de currículos, utilizando processamento de linguagem natural e um sistema de pontuação objetivo.

## 2. Componentes Principais

### 2.1 Importações e Configuração
```python
import os, json, re
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv
```
- Utiliza `dotenv` para gerenciamento seguro de credenciais
- Integra com a API OpenAI via `langchain_openai`
- Implementa fluxo de trabalho usando `langgraph`

### 2.2 Estruturas de Dados
#### Candidate (TypedDict)
```python
class Candidate(TypedDict):
    name: str
    resume: str
    skills: List[str]
    experience: int
    education_level: str
    score: float
    feedback: str
```

#### State (TypedDict)
```python
class State(TypedDict):
    candidates: List[Candidate]
    job_requirements: Dict[str, any]
    result: str
```

## 3. Fluxo de Processamento

### 3.1 Extração de Informações
- Função `extract_information`
- Utiliza regex para extrair dados estruturados dos currículos:
  - Nome do candidato
  - Experiência profissional
  - Formação acadêmica
  - Habilidades técnicas

### 3.2 Avaliação de Candidatos
Sistema de pontuação baseado em três critérios:
1. **Habilidades Técnicas** (4 pontos)
   - Correspondência com requisitos da vaga
   - Pontuação proporcional às habilidades encontradas

2. **Experiência Profissional** (3 pontos)
   - Avaliação binária baseada na experiência mínima
   - Pontuação total se atender requisito

3. **Formação Acadêmica** (3 pontos)
   - Avaliação binária da formação relevante
   - Pontuação total se compatível

### 3.3 Classificação
- Ordenação por pontuação total (0-10)
- Critérios de desempate:
  1. Maior pontuação em habilidades
  2. Maior tempo de experiência

### 3.4 Geração de Relatório
Relatório estruturado incluindo:
- Requisitos da vaga
- Detalhes do melhor candidato
- Ranking completo com pontuações
- Feedback detalhado por candidato

## 4. Workflow (LangGraph)
```python
workflow = StateGraph(State)
workflow.add_node("extract_information", extract_information)
workflow.add_node("evaluate_candidate", evaluate_candidate)
workflow.add_node("rank_candidates", rank_candidates)
workflow.add_node("generate_result", generate_result)
```

## 5. Considerações de Segurança e Privacidade
- Conformidade com LGPD
- Tratamento seguro de dados sensíveis
- Validação robusta de entradas
- Tratamento adequado de exceções

## 6. Possíveis Extensões
1. Integração com bancos de dados
2. Interface de usuário web
3. Critérios adicionais de avaliação
4. Análise de soft skills
5. Integração com sistemas de RH

## 7. Exemplo de Uso
```python
candidates = [
    {"resume": "..."},  # Lista de currículos
]

job_requirements = {
    "skills_required": ["Python", "SQL", "Web"],
    "minimum_experience": 3,
    "education_level": "Graduação",
    "desired_attributes": ["Trabalho em equipe"]
}

result = process_candidates(candidates, job_requirements)
```

## 8. Limitações e Melhorias Futuras
- Aprimoramento da extração de informações
- Análise semântica mais sofisticada
- Suporte a múltiplos idiomas
- Personalização por setor/indústria

Este sistema oferece uma solução robusta e objetiva para o processo de triagem de currículos, combinando técnicas de processamento de texto com um sistema de pontuação transparente e auditável.

In [1]:
!pip install langgraph langchain-openai langchain-core python-dotenv typing_extensions openai


## Inicio do script, importando as biliotecas necessarias e etc. 

In [2]:
import os
import json
import re
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Definir a chave da API
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Inicializar o modelo de linguagem
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Definir a estrutura de dados do candidato
class Candidate(TypedDict):
    name: str
    resume: str
    skills: List[str]
    experience: int
    education_level: str
    score: float
    feedback: str

# Definir o estado do fluxo de trabalho
class State(TypedDict):
    candidates: List[Candidate]
    job_requirements: Dict[str, any]
    result: str

def extract_information_from_resume(resume_text: str) -> Dict:
    """Extrai informações do currículo usando regex."""
    info = {}
    
    # Extrair nome
    name_match = re.search(r'Nome:\s*(.*?)(?:\n|$)', resume_text)
    info['name'] = name_match.group(1).strip() if name_match else ''
    
    # Extrair experiência
    exp_match = re.search(r'(\d+)\s*anos? de experiência', resume_text)
    info['experience'] = int(exp_match.group(1)) if exp_match else 0
    
    # Extrair educação
    edu_patterns = [
        r'graduação em ([^\.]*)',
        r'formad[ao] em ([^\.]*)',
        r'mestrado em ([^\.]*)'
    ]
    info['education_level'] = ''
    for pattern in edu_patterns:
        edu_match = re.search(pattern, resume_text, re.IGNORECASE)
        if edu_match:
            info['education_level'] = edu_match.group(1).strip()
            break
    
    # Extrair habilidades
    skills = []
    skill_patterns = [
        r'proficiente em ([^\.]*)',
        r'conhecimento em ([^\.]*)',
        r'especialista em ([^\.]*)',
        r'experiência (?:com|em) ([^\.]*)'
    ]
    for pattern in skill_patterns:
        skill_match = re.search(pattern, resume_text, re.IGNORECASE)
        if skill_match:
            skills.extend([s.strip() for s in skill_match.group(1).split(',')])
            skills.extend([s.strip() for s in skill_match.group(1).split(' e ')])
    
    info['skills'] = list(set(skills))  # Remove duplicatas
    return info

def extract_information(state: State) -> State:
    """Extrai informações dos currículos."""
    updated_candidates = []
    
    for candidate in state['candidates']:
        try:
            info = extract_information_from_resume(candidate['resume'])
            candidate.update({
                'name': info['name'],
                'skills': info['skills'],
                'experience': info['experience'],
                'education_level': info['education_level'],
                'score': 0.0,
                'feedback': ''
            })
        except Exception as e:
            print(f"Erro ao extrair informações: {e}")
        
        updated_candidates.append(candidate)
    
    state['candidates'] = updated_candidates
    return state

In [3]:
def evaluate_candidate(state: State) -> State:
    """Avalia os candidatos com base nos requisitos."""
    updated_candidates = []
    
    for candidate in state['candidates']:
        try:
            # Sistema de pontuação direto
            score = 0
            feedback_points = []
            
            # Avaliar habilidades (4 pontos)
            required_skills = state['job_requirements']['skills_required']
            matching_skills = []
            for req_skill in required_skills:
                for candidate_skill in candidate['skills']:
                    if req_skill.lower() in candidate_skill.lower():
                        matching_skills.append(req_skill)
                        break
            
            skill_score = len(set(matching_skills)) / len(required_skills) * 4
            score += skill_score
            feedback_points.append(f"Possui {len(set(matching_skills))} de {len(required_skills)} habilidades requeridas")
            
            # Avaliar experiência (3 pontos)
            min_exp = state['job_requirements']['minimum_experience']
            if candidate['experience'] >= min_exp:
                score += 3
                feedback_points.append(f"Experiência adequada ({candidate['experience']} anos)")
            else:
                feedback_points.append(f"Experiência abaixo do requisito ({candidate['experience']} anos)")
            
            # Avaliar educação (3 pontos)
            if candidate['education_level']:
                score += 3
                feedback_points.append(f"Possui formação adequada: {candidate['education_level']}")
            
            # Atualizar candidato
            candidate['score'] = min(score, 10)  # Limita a 10 pontos
            candidate['feedback'] = ". ".join(feedback_points)
            
        except Exception as e:
            candidate['score'] = 0.0
            candidate['feedback'] = f"Erro na avaliação: {str(e)}"
        
        updated_candidates.append(candidate)
    
    state['candidates'] = updated_candidates
    return state

def rank_candidates(state: State) -> State:
    """Classifica os candidatos por pontuação."""
    try:
        state['candidates'].sort(key=lambda x: (float(x.get('score', 0)), x.get('experience', 0)), reverse=True)
    except Exception as e:
        print(f"Erro ao classificar candidatos: {e}")
    return state

In [4]:
def generate_result(state: State) -> State:
    """Gera o relatório final."""
    try:
        if not state['candidates']:
            state['result'] = 'Não há candidatos para avaliar.'
            return state

        report = []
        report.append("=== RELATÓRIO DE CLASSIFICAÇÃO DE CANDIDATOS ===\n")
        
        # Requisitos da vaga
        report.append("Requisitos da Vaga:")
        report.append(f"- Habilidades: {', '.join(state['job_requirements']['skills_required'])}")
        report.append(f"- Experiência Mínima: {state['job_requirements']['minimum_experience']} anos")
        report.append(f"- Educação: {state['job_requirements']['education_level']}")
        report.append(f"- Atributos Desejados: {', '.join(state['job_requirements']['desired_attributes'])}\n")
        
        # Melhor candidato
        best_candidate = state['candidates'][0]
        report.append("MELHOR CANDIDATO:")
        report.append(f"Nome: {best_candidate.get('name', 'Nome não disponível')}")
        report.append(f"Pontuação: {best_candidate.get('score', 0):.1f}/10")
        report.append(f"Experiência: {best_candidate.get('experience', 0)} anos")
        report.append(f"Educação: {best_candidate.get('education_level', 'Não informado')}")
        report.append(f"Habilidades: {', '.join(best_candidate.get('skills', []))}")
        report.append(f"Feedback: {best_candidate.get('feedback', 'Sem feedback disponível')}\n")
        
        # Ranking completo
        report.append("RANKING COMPLETO:")
        for i, candidate in enumerate(state['candidates'], 1):
            report.append(f"{i}. {candidate.get('name', f'Candidato {i}')} - Pontuação: {candidate.get('score', 0):.1f}/10")
        
        state['result'] = '\n'.join(report)
    except Exception as e:
        state['result'] = f"Erro ao gerar relatório: {str(e)}"
    
    return state

# Construir o workflow
workflow = StateGraph(State)

# Adicionar nós
workflow.add_node("extract_information", extract_information)
workflow.add_node("evaluate_candidate", evaluate_candidate)
workflow.add_node("rank_candidates", rank_candidates)
workflow.add_node("generate_result", generate_result)

# Definir as arestas
workflow.add_edge(START, "extract_information")
workflow.add_edge("extract_information", "evaluate_candidate")
workflow.add_edge("evaluate_candidate", "rank_candidates")
workflow.add_edge("rank_candidates", "generate_result")
workflow.add_edge("generate_result", END)

# Compilar o grafo
app = workflow.compile()

def process_candidates(candidates: List[Candidate], job_requirements: Dict[str, any]) -> str:
    """Processa os candidatos e retorna o resultado."""
    try:
        initial_state = State(
            candidates=candidates,
            job_requirements=job_requirements,
            result=""
        )
        
        final_state = app.invoke(initial_state)
        return final_state.get('result', 'Erro: Resultado não disponível')
    except Exception as e:
        return f"Erro durante o processamento: {str(e)}"

# Exemplo de uso
if __name__ == "__main__":
    # Exemplo de candidatos
    candidates = [
        {
            "resume": """
            Nome: Maria Silva
            
            Maria é desenvolvedora de software com 5 anos de experiência. 
            Ela é proficiente em Python, JavaScript e tem experiência com bancos de dados SQL. 
            Possui graduação em Ciência da Computação.
            Trabalhou em projetos de desenvolvimento web e está familiarizada com metodologias ágeis.
            """
        },
        {
            "resume": """
            Nome: João Souza
            
            João é engenheiro de software com 8 anos de experiência. 
            Especialista em Java e C++. 
            Possui mestrado em Engenharia de Software.
            Tem experiência em liderança de equipes e desenvolvimento de sistemas de grande escala.
            """
        },
        {
            "resume": """
            Nome: Ana Pereira
            
            Ana é recém-formada em Sistemas de Informação. 
            Possui conhecimento em Python, HTML e CSS. 
            Participou de projetos acadêmicos relacionados a desenvolvimento web.
            Está em busca de sua primeira oportunidade profissional.
            """
        }
    ]

    # Requisitos da vaga
    job_requirements = {
        "skills_required": ["Python", "SQL", "Desenvolvimento Web"],
        "minimum_experience": 3,
        "education_level": "Graduação em área relacionada",
        "desired_attributes": ["Trabalho em equipe", "Metodologias Ágeis"]
    }

    # Processar os candidatos
    result = process_candidates(candidates, job_requirements)
    print(result)

=== RELATÓRIO DE CLASSIFICAÇÃO DE CANDIDATOS ===

Requisitos da Vaga:
- Habilidades: Python, SQL, Desenvolvimento Web
- Experiência Mínima: 3 anos
- Educação: Graduação em área relacionada
- Atributos Desejados: Trabalho em equipe, Metodologias Ágeis

MELHOR CANDIDATO:
Nome: Maria Silva
Pontuação: 8.7/10
Experiência: 5 anos
Educação: Ciência da Computação
Habilidades: Python, tem experiência com bancos de dados SQL, Python, JavaScript, JavaScript e tem experiência com bancos de dados SQL, bancos de dados SQL
Feedback: Possui 2 de 3 habilidades requeridas. Experiência adequada (5 anos). Possui formação adequada: Ciência da Computação

RANKING COMPLETO:
1. Maria Silva - Pontuação: 8.7/10
2. João Souza - Pontuação: 6.0/10
3. Ana Pereira - Pontuação: 4.3/10




# Sistema de Pontuação para Avaliação de Candidatos

O sistema de pontuação foi projetado para avaliar candidatos em uma escala de 0 a 10 pontos, considerando três critérios principais:

## 1. Habilidades Técnicas (4 pontos)
- **Pontuação máxima:** 4.0 pontos
- **Método de cálculo:** 
  ```python
  skill_score = (número_de_habilidades_correspondentes / número_de_habilidades_requeridas) * 4
  ```
- **Exemplo:**
  - Se a vaga requer 3 habilidades (Python, SQL, Web)
  - E o candidato possui 2 delas
  - Então: (2/3) * 4 = 2.67 pontos

## 2. Experiência Profissional (3 pontos)
- **Pontuação máxima:** 3.0 pontos
- **Método de cálculo:**
  ```python
  if experiência_candidato >= experiência_mínima:
      experience_score = 3
  else:
      experience_score = 0
  ```
- **Característica:** Pontuação binária (tudo ou nada)
- **Exemplo:**
  - Se a vaga requer 3 anos de experiência
  - E o candidato tem 5 anos
  - Então: 3.0 pontos

## 3. Formação Acadêmica (3 pontos)
- **Pontuação máxima:** 3.0 pontos
- **Método de cálculo:**
  ```python
  if possui_formação_relevante:
      education_score = 3
  else:
      education_score = 0
  ```
- **Característica:** Pontuação binária (tudo ou nada)
- **Exemplo:**
  - Se o candidato possui formação na área
  - Então: 3.0 pontos

## Pontuação Final
- **Cálculo:** Soma dos três critérios
- **Limite:** Máximo de 10 pontos
```python
score_final = min(skill_score + experience_score + education_score, 10)
```

## Feedback
Para cada avaliação, o sistema gera um feedback detalhado incluindo:
1. Número de habilidades correspondentes
2. Status da experiência profissional
3. Status da formação acadêmica

## Exemplo de Avaliação Completa
```markdown
Candidato: Maria Silva
- Habilidades: 3/3 = 4.0 pontos
- Experiência: 5 anos (≥ 3 anos) = 3.0 pontos
- Formação: Graduação em Ciência da Computação = 3.0 pontos
Pontuação Final: 10.0/10
```

## Critérios de Desempate
Em caso de empate na pontuação final, o sistema considera:
1. Maior pontuação em habilidades técnicas
2. Maior tempo de experiência profissional

Este sistema foi projetado para ser:
- Objetivo
- Transparente
- Facilmente auditável
- Consistente em suas avaliações